In [1]:
!pip install unstructured[md] sentence-transformers langchain chromadb

In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings,HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter,MarkdownHeaderTextSplitter,CharacterTextSplitter
from langchain.chains import VectorDBQA
from langchain.document_loaders import TextLoader,DirectoryLoader

In [24]:
# Load and process the text
loader = TextLoader('./merged/merged.md')
document = loader.load()

text_splitter = RecursiveCharacterTextSplitter(        
    chunk_size = 400,
    chunk_overlap  = 30,
    length_function = len,
    is_separator_regex = True,
)
splits = text_splitter.split_documents(document)
len(splits)

508

In [39]:
#persist_directory = 'db'

model_name = "./servable/encoders/default/transformer/"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

vectordb = Chroma.from_documents(documents=splits, embedding=hf) #persist_directory=persist_directory)


InvalidDimensionException: Embedding dimension 384 does not match collection dimensionality 1024

In [58]:
#vectordb.persist()
#vectordb = None
#vectordb = Chroma(persist_directory=persist_directory, embedding_function=hf)

In [17]:
result = vectordb.similarity_search_with_relevance_scores("Что такое теншен?",k=10)
print(result)

[(Document(page_content='СМАРТОКРАТИЯ\n\n#', metadata={'source': './merged/merged.md'}), 0.16649001538855113), (Document(page_content='СМАРТОКРАТИЯ\n\n#', metadata={'source': './merged/merged.md'}), 0.1664899310948541), (Document(page_content='СМАРТОКРАТИЯ\n\n#', metadata={'source': './merged/merged.md'}), 0.1664899310948541), (Document(page_content='Обязательные роли Круга\n\n##', metadata={'source': './merged/merged.md'}), 0.14455064198359924), (Document(page_content='ПРИЛОЖЕНИЯ\n\n#', metadata={'source': './merged/merged.md'}), 0.11135409822247866), (Document(page_content='ПРИЛОЖЕНИЯ\n\n#', metadata={'source': './merged/merged.md'}), 0.11135409822247866), (Document(page_content='ПРИЛОЖЕНИЯ\n\n#', metadata={'source': './merged/merged.md'}), 0.11135401392878164), (Document(page_content='ПРИЛОЖЕНИЯ\n\n#', metadata={'source': './merged/merged.md'}), 0.11135401392878164), (Document(page_content='Операционная система управления**\n\n##', metadata={'source': './merged/merged.md'}), 0.08775

In [21]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from tqdm.auto import tqdm, trange
model_name = 'doc2query/msmarco-russian-mt5-base-v1'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def create_queries(para):
    input_ids = tokenizer.encode(para, return_tensors='pt')
    with torch.no_grad():
        # Here we use top_k / top_k random sampling. It generates more diverse queries, but of lower quality
        sampling_outputs = model.generate(
            input_ids=input_ids,
            max_length=96,
            do_sample=True,
            top_p=0.95,
            top_k=10, 
            num_return_sequences=5
            )
        
        # Here we use Beam-search. It generates better quality queries, but with less diversity
        beam_outputs = model.generate(
            input_ids=input_ids, 
            max_length=96, 
            num_beams=5, 
            no_repeat_ngram_size=2, 
            num_return_sequences=5, 
            early_stopping=True
        )

    
    queries = []
    for i in range(len(beam_outputs)):
        queries.append(tokenizer.decode(beam_outputs[i], skip_special_tokens=True))

    return queries


In [25]:
qs = []
from tqdm.auto import tqdm

for split in tqdm(splits):
    qs.append(create_queries(str(split)))

  0%|          | 0/508 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [31]:
import json
from typing import List, Dict, Any
from torch.utils.data import Dataset
from quaterion.dataset.similarity_samples import SimilarityPairSample

class SimilarityPairSample:
    obj_a: Any  # question
    obj_b: Any  # answer
    score: float = 1.0  # Measure of similarity. Usually converted to bool
    # Consider all examples outside this group as negative samples.
    # By default, all samples belong to group 0 - therefore other samples could not be used as negative examples.
    subgroup: int = 0

import json
from typing import List, Dict
from torch.utils.data import Dataset
from quaterion.dataset.similarity_samples import SimilarityPairSample


class FAQDataset(Dataset):

    def __init__(self, splits,qs):
        self.splits = splits
        self.qs = qs

    def __getitem__(self, index) -> SimilarityPairSample:
        question = qs[index][0]
        answer = splits[index]
        
        # All questions have a unique subgroup
        # Meaning that all other answers are considered negative pairs
        subgroup = hash(question)
        score = 1
        return SimilarityPairSample(
            obj_a=question, obj_b=answer, score=score, subgroup=subgroup
        )

    def __len__(self):
        return len(splits)


In [32]:
import os
from torch import Tensor, nn
from sentence_transformers.models import Transformer, Pooling
from quaterion_models.types import TensorInterchange, CollateFnType
from quaterion_models.encoders import Encoder


class FAQEncoder(Encoder):
    def __init__(self, transformer, pooling):
        super().__init__()
        self.transformer = transformer
        self.pooling = pooling
        self.encoder = nn.Sequential(self.transformer, self.pooling)

    @property
    def trainable(self) -> bool:
        # Defines if we want to train encoder itself, or head layer only
        return False

    @property
    def embedding_size(self) -> int:
        return self.transformer.get_word_embedding_dimension()

    def forward(self, batch: TensorInterchange) -> Tensor:
        return self.encoder(batch)["sentence_embedding"]

    def get_collate_fn(self) -> CollateFnType:
        # `collate_fn` is a function that converts input samples into Tensor(s) for use as encoder input.
        return self.transformer.tokenize

    @staticmethod
    def _transformer_path(path: str) -> str:
        # just an additional method to reduce amount of repeated code
        return os.path.join(path, "transformer")

    @staticmethod
    def _pooling_path(path: str) -> str:
        return os.path.join(path, "pooling")

    def save(self, output_path: str):
        # to provide correct saving of encoder layers we need to implement it manually
        transformer_path = self._transformer_path(output_path)
        os.makedirs(transformer_path, exist_ok=True)

        pooling_path = self._pooling_path(output_path)
        os.makedirs(pooling_path, exist_ok=True)

        self.transformer.save(transformer_path)
        self.pooling.save(pooling_path)

    @classmethod
    def load(cls, input_path: str) -> Encoder:
        transformer = Transformer.load(cls._transformer_path(input_path))
        pooling = Pooling.load(cls._pooling_path(input_path))
        return cls(transformer=transformer, pooling=pooling)


In [34]:
from quaterion.eval.attached_metric import AttachedMetric
from torch.optim import Adam
from quaterion import TrainableModel
from quaterion.train.cache import CacheConfig, CacheType
from quaterion.loss import MultipleNegativesRankingLoss
from sentence_transformers import SentenceTransformer
from quaterion.eval.pair import RetrievalPrecision, RetrievalReciprocalRank
from sentence_transformers.models import Transformer, Pooling
from quaterion_models.heads.skip_connection_head import SkipConnectionHead


class FAQModel(TrainableModel):
    def __init__(self, lr=10e-5, *args, **kwargs):
        self.lr = lr
        super().__init__(*args, **kwargs)

    def configure_metrics(self):
        # attach batch-wise metrics which will be automatically computed and logged during training
        return [
            AttachedMetric(
                "RetrievalPrecision",
                RetrievalPrecision(k=1),
                prog_bar=True,
                on_epoch=True,
            ),
            AttachedMetric(
                "RetrievalReciprocalRank",
                RetrievalReciprocalRank(),
                prog_bar=True,
                on_epoch=True
            ),
        ]

    def configure_optimizers(self):
        return Adam(self.model.parameters(), lr=self.lr)

    def configure_loss(self):
        # `symmetric` means that we take into account correctness of both the closest answer to a question and the closest question to an answer
        return MultipleNegativesRankingLoss(symmetric=True)

    def configure_encoders(self):
        pre_trained_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
        transformer: Transformer = pre_trained_model[0]
        pooling: Pooling = pre_trained_model[1]
        encoder = FAQEncoder(transformer, pooling)
        return encoder

    def configure_head(self, input_embedding_size: int):
        return SkipConnectionHead(input_embedding_size)

    def configure_caches(self):
        # Cache stores frozen encoder embeddings to prevent repeated calculations and increase training speed.
        # AUTO preserves the current encoder's device as storage, batch size does not affect training and is used only to fill the cache before training.
        return CacheConfig(CacheType.AUTO, batch_size=256)


In [37]:
import os

import torch
import pytorch_lightning as pl

from quaterion import Quaterion
from quaterion.dataset import PairsSimilarityDataLoader
from quaterion.eval.evaluator import Evaluator
from quaterion.eval.pair import RetrievalReciprocalRank, RetrievalPrecision
from quaterion.eval.samplers.pair_sampler import PairSampler

DATA_DIR = 'data'


def run(model, params):
    use_gpu = params.get("cuda", torch.cuda.is_available())

    trainer = pl.Trainer(
        min_epochs=params.get("min_epochs", 1),
        max_epochs=params.get("max_epochs", 300),  # cache makes it possible to use a huge amount of epochs
        auto_select_gpus=use_gpu,
        log_every_n_steps=params.get("log_every_n_steps", 10),  # increase to speed up training
        gpus=int(use_gpu),
        num_sanity_val_steps=2,
    )
    train_dataset = FAQDataset(splits[:400],qs[:400])
    val_dataset = FAQDataset(splits[400:],qs[400:])
    train_dataloader = PairsSimilarityDataLoader(train_dataset, batch_size=128)
    val_dataloader = PairsSimilarityDataLoader(val_dataset, batch_size=128)
    Quaterion.fit(model, trainer, train_dataloader, val_dataloader)

    metrics = {
        "rrk": RetrievalReciprocalRank(),
        "rp@1": RetrievalPrecision(k=1)
    }
    sampler = PairSampler()
    evaluator = Evaluator(metrics, sampler)
    results = Quaterion.evaluate(evaluator, val_dataset, model.model)  # calculate metrics on the whole dataset to obtain more representative metrics values
    print(f"results: {results}")


# launch training
pl.seed_everything(42, workers=True)
faq_model = FAQModel()
run(faq_model {})
faq_model.save_servable("servable")


Global seed set to 42
/home/kglin/miniconda3/envs/central_bank/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:478: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
/home/kglin/miniconda3/envs/central_bank/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:589: LightningDeprecationWarning: The Trainer argument `auto_select_gpus` has been deprecated in v1.9.0 and will be removed in v2.0.0. Please use the function `pytorch_lightning.accelerators.find_usable_cuda_devices` instead.
  rank_zero_deprecation(
Auto select gpus: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/kglin/miniconda3/envs/central_bank/lib/python3.11/site-packages/pytorch_lightning/train

Predicting: 0it [00:00, ?it/s]

2023-09-30 17:10:48.876 | DEBUG    | quaterion.train.cache_mixin:_cache:223 - Caching has been successfully finished
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type                         | Params
--------------------------------------------------------
0 | _model | SimilarityModel              | 117 M 
1 | _loss  | MultipleNegativesRankingLoss | 0     
--------------------------------------------------------
148 K     Trainable params
117 M     Non-trainable params
117 M     Total params
471.208   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/kglin/miniconda3/envs/central_bank/lib/python3.11/site-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (4) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=300` reached.
64it [00:22,  2.79it/s]                                                         


results: {'rrk': tensor(0.1833), 'rp@1': tensor(0.0945)}


In [42]:
!jupyter labextension install @jupyter-widgets/jupyterlab-manager
!jupyter labextension install node

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
(Deprecated) Installing extensions with the jupyter labextension install command is now deprecated and will be removed in a future major version of JupyterLab.

Users should manage prebuilt extensions with package managers like pip and conda, and extension authors are encouraged to distribute their extensions as prebuilt packages 
/home/kglin/miniconda3/envs/central_bank/lib/python3.11/site-packages/jupyterlab/debuglog.py:56: UserWarning: An error occurred.
  warnings.warn("An error occurred.")
/home/kglin/miniconda3/envs/central_bank/lib/python3.11/site-packages/jupyterlab/debuglog.py:57: UserWarning: ValueError: Please install nodejs >=18.0.0 before continuing. node